In [1]:
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1
%env OPENBLAS_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1


env: OMP_NUM_THREADS=1
env: MKL_NUM_THREADS=1
env: OPENBLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1


In [ ]:
"""
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=6,  # or 32, depending on workload
    threads_per_worker=2,  # isolate GIL
    memory_limit="auto",  # or tune to your RAM
    dashboard_address=":8788",
)
client = Client(cluster)
print(cluster.dashboard_link)
"""


In [2]:
from zarrnii import ZarrNii

In [3]:
img = ZarrNii.from_ome_zarr('/nfs/khan/datasets/MIND/mouse_app_lecanemab/sub-AS40F2/micr/sub-AS40F2_sample-brain_acq-imaris4x_SPIM.ome.zarr',channels=[0])
img                            


ZarrNii(
  name='image', shape=(1, 2080, 10112, 5760), dims=['c', 'z', 'y', 'x'],
  axes_order='ZYX', xyz_orientation='LAI',
  scale={'c': 1, 'z': 0.00275, 'y': 0.001625, 'x': 0.001625},
  dtype=uint16, chunksize=(1, 1, 4096, 4096)
)

In [4]:
cropped = img.crop((0,0,1000),(5760,10112,1010))
cropped.data = cropped.data.rechunk((1,1,cropped.data.shape[-2],cropped.data.shape[-1]))
cropped

ZarrNii(
  name='image', shape=(1, 10, 10112, 5760), dims=['c', 'z', 'y', 'x'],
  axes_order='ZYX', xyz_orientation='LAI',
  scale={'c': 1, 'z': 0.00275, 'y': 0.001625, 'x': 0.001625},
  dtype=uint16, chunksize=(1, 1, 10112, 5760)
)

In [5]:
destriped = cropped.destripe()

In [6]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    destriped.to_ome_zarr('test_destriped.ome.zarr') #took 338.20 s with threaded scheduler, setting blas threads to 1

[########################################] | 100% Completed | 341.05 s


In [ ]:
cropped.to_nifti('test_crop.nii')

In [7]:
ZarrNii.from_ome_zarr('test_destriped.ome.zarr').to_nifti('test_destriped.nii')

'test_destriped_bgthresh1.nii'